In [ ]:
import json
from datetime import datetime

with open('./sources/data.json') as f:
    data = json.load(f)
result = []
for date, value in data:
    date = datetime.utcfromtimestamp(int(str(date).rstrip('0'))).date()
    value = int(value)
    print(date, value)
    result.append({'date': str(date), 'value': value})

# ## Write the result to a new JSON file
with open('./sources/data_edited_date_utc.json', 'w') as f:
    json.dump(result, f)

In [ ]:
import json
from datetime import datetime

with open('./sources/data.json') as f:
    data = json.load(f)
result = []
for date, value in data:
    # date = int(str(date).rstrip('0'))
    value = int(value)
    # print(date, value)
    result.append({'date': str(date), 'value': value, 'value2': int(value * 0.2)})

# ## Write the result to a new JSON file
with open('./sources/data_edited_value2.json', 'w') as f:
    json.dump(result, f)

In [3]:

import duckdb
from pathlib import Path

def concat_parquet_to_duckdb(folder_path, output_file='sec.duckdb'):
    files = list(Path(folder_path).glob('*.parquet'))
    if Path(output_file).suffix != '.duckdb':
        output_file += '.duckdb'

    con = duckdb.connect(database=output_file)
    cur = con.cursor()

    # create and populate 'all_data' table
    cur.execute("""CREATE TABLE IF NOT EXISTS main AS SELECT 
                    accession_number,
                    cik,
                    cik_name,
                    cik_ticker,
                    cik_ticker_name,
                    cusip,
                    name_of_issuer,
                    cusip_ticker,
                    cusip_ticker_name,
                    quarter, 
                    rdate,    
                    shares,
                    value,
                    tr_id,
                    tr_type,
                    tr_value,
                    tr_shares,
                    put_call
    FROM parquet_scan('{}')""".format(files[0]))

    # cur.execute("CREATE INDEX IF NOT EXISTS idx_accession_number ON main (cik, quarter, cusip, accession_number)")
    
    for file in files[1:]:
        cur.execute("""INSERT INTO main SELECT 
        accession_number,
        cik,
        cik_name,
        cik_ticker,
        cik_ticker_name,
        cusip,
        name_of_issuer,
        cusip_ticker,
        cusip_ticker_name,
        quarter, 
        rdate,    
        shares,
        value,
        tr_id,
        tr_type,
        tr_value,
        tr_shares,
        put_call
    FROM parquet_scan('{}')""".format(file))

    # create and populate 'all_cik_quarter_cusip' table
    cur.execute("""
        CREATE TABLE IF NOT EXISTS all_cik_quarter_cusip AS 
        SELECT     
            cik,
            quarter,
            cusip,
            ANY_VALUE(cusip_ticker_name) AS cusip_ticker_name,
            ANY_VALUE(cik_name) as cik_name,
            SUM(value) AS value_usd,
            SUM(shares) AS shares,
            ROUND(SUM(value) / SUM(shares), 2) as avg_price_usd,
            ANY_VALUE(cusip_ticker) as cusip_ticker,
            ANY_VALUE(name_of_issuer) as name_of_issuer,
            ROUND(SUM(value) / SUM(SUM(value)) OVER (PARTITION BY cik, quarter), 2) AS pct_pct
    FROM main.main
    WHERE accession_number != 'SYNTHETIC-CLOSE' AND quarter > '1998Q1'  
    GROUP BY cik, quarter, cusip
    ORDER BY 1, 2, 6 DESC
    """)

    cur.close()
    con.close()
    
# call the function
# concat_parquet_to_duckdb('/Users/yo_macbook/Documents/app_data/test_cik_consolidated_to')


In [4]:
%%time
# parquet_folder = Path(r'/Users/yo_macbook/Documents/app_data/test_parq')

parquet_folder = Path(r'/Users/yo_macbook/Documents/app_data/TR_04_CONS_CIK_CUSIP')
output_file = r'/Users/yo_macbook/Documents/app_data/TR_05_TEST_FINAL_DB_PARQ_ARROW/sec.duckdb'

concat_parquet_to_duckdb(parquet_folder, output_file='sec_full_tr.duckdb')

#  12 sec, - 759 Mb from 83 parquet files of 110 MB

CPU times: user 11min 3s, sys: 4min, total: 15min 3s
Wall time: 8min 32s


In [2]:
import duckdb
import polars
con = duckdb.connect(database=r'/Users/yo_macbook/Documents/app_data/TR_05_TEST_FINAL_DB_PARQ_ARROW/sec.duckdb')
cur = con.cursor()
# print(cur.execute("""SELECT * FROM information_schema.columns WHERE table_name = 'all_cik_quarter_cusip'""").pl().head(4))
print(cur.execute("""SELECT * FROM main limit 3""").pl().columns)
cur.close()
con.close()

['accession_number', 'cik', 'cik_name', 'cik_ticker', 'cik_ticker_name', 'cusip', 'name_of_issuer', 'cusip_ticker', 'cusip_ticker_name', 'quarter', 'rdate', 'shares', 'value']


In [2]:
!pip install polars

  Obtaining dependency information for polars from https://files.pythonhosted.org/packages/ba/55/79665e08ceaf8904ecd4864eec3fae67c34a06b7fa88d931b78528e610a4/polars-0.18.11-cp38-abi3-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 13.8 MB/s eta 0:00:00m eta 0:00:010:01:01

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# accession_number,
# cik,
# cik_name,
# cik_ticker,
# cik_ticker_name,
# cusip, 
# cusip_ticker,
# cusip_ticker_name,
# quarter, 
# rdate,    
# shares,
# value 

In [5]:

import duckdb
from pathlib import Path

def concat_parquet_to_duckdb2(folder_path, output_file='sec_full.duckdb'):
    files = list(Path(folder_path).glob('*.parquet'))
    if Path(output_file).suffix != '.duckdb':
        output_file += '.duckdb'

    con = duckdb.connect(database=output_file)
    cur = con.cursor()

    # create and populate 'all_data' table
    cur.execute("""CREATE OR REPLACE TABLE main AS SELECT 
                    accession_number,
                    cik,
                    cik_name,
                    cik_ticker,
                    cik_ticker_name,
                    cusip,
                    name_of_issuer,
                    cusip_ticker,
                    cusip_ticker_name,
                    quarter, 
                    rdate,
                    fdate,
                    shares,
                    value 
    FROM parquet_scan('{}')""".format(files[0]))

    # cur.execute("CREATE INDEX idx_on_cik_and_quarter ON main(cik, quarter)")

    ## insert data
    for file in files[1:]:
        cur.execute("""INSERT INTO main SELECT 
        accession_number,
        cik,
        cik_name,
        cik_ticker,
        cik_ticker_name,
        cusip,
        name_of_issuer,
        cusip_ticker,
        cusip_ticker_name,
        quarter, 
        rdate,  
        fdate,
        shares,
        value 
    FROM parquet_scan('{}')""".format(file))


    cur.close()
    con.close()
    
# call the function
# concat_parquet_to_duckdb('/Users/yo_macbook/Documents/app_data/test_cik_consolidated_to')


In [6]:
%%time
parquet_folder = Path(r'/Users/yo_macbook/Documents/app_data/TR_04_CONS_CIK_CUSIP')
output_file = r'/Users/yo_macbook/Documents/app_data/TR_05_TEST_FINAL_DB_PARQ_ARROW/sec_full.duckdb'

concat_parquet_to_duckdb2(parquet_folder, output_file) 

#  12 sec, - 759 Mb from 83 parquet files of 110 MB
# 12 min - with index on cik and quarter - from all files 

CPU times: user 3min 46s, sys: 14.4 s, total: 4min
Wall time: 3min 50s


In [ ]:
import duckdb
con = duckdb.connect(database=r'/Users/yo_macbook/Documents/app_data/TR_05_TEST_FINAL_DB_PARQ_ARROW/sec_full.duckdb')
cur = con.cursor()
cur.close()
con.close()

# SEC_FULL with .arrow

In [24]:
import duckdb
import pyarrow as pa
import pandas as pd
from pathlib import Path

def concat_parquet_to_arrow(folder_path, output_file='sec_full.arrow'):
    files = list(Path(folder_path).glob('*.parquet'))
    if Path(output_file).suffix != '.arrow':
        output_file += '.arrow'

    con = duckdb.connect(database=':memory:')  # Use in-memory DB for temporary processing
    cur = con.cursor()

    # create and populate 'all_data' table
    cur.execute("""CREATE OR REPLACE TABLE main AS SELECT 
                    accession_number,
                    cik,
                    cik_name,
                    cik_ticker,
                    cik_ticker_name,
                    cusip,
                    name_of_issuer,
                    cusip_ticker,
                    cusip_ticker_name,
                    quarter, 
                    rdate,
                    fdate,
                    shares,
                    value 
    FROM parquet_scan('{}')""".format(files[0]))

    # cur.execute("CREATE INDEX idx_on_cik_and_quarter ON main(cik, quarter)")

    ## insert data
    for file in files[1:]:
        cur.execute("""INSERT INTO main SELECT 
        accession_number,
        cik,
        cik_name,
        cik_ticker,
        cik_ticker_name,
        cusip,
        name_of_issuer,
        cusip_ticker,
        cusip_ticker_name,
        quarter, 
        rdate,  
        fdate,
        shares,
        value 
    FROM parquet_scan('{}')""".format(file))

    # Fetch data into a Pandas DataFrame
    df = cur.execute("SELECT * FROM main").fetchdf()

    # Convert the DataFrame to Arrow format and save it
    table = pa.Table.from_pandas(df)
    with pa.OSFile(output_file, 'wb') as sink:
        with pa.RecordBatchFileWriter(sink, table.schema) as writer:
            writer.write_table(table)

    cur.close()
    con.close()

In [25]:


%%time
parquet_folder = Path(r'/Users/yo_macbook/Documents/app_data/TR_04_CONS_CIK_CUSIP')
output_file = r'/Users/yo_macbook/Documents/app_data/TR_05_TEST_FINAL_DB_PARQ_ARROW/sec_full.arrow'

concat_parquet_to_arrow(parquet_folder, output_file) 

CPU times: user 2min 35s, sys: 3min 13s, total: 5min 49s
Wall time: 7min 41s
